In [39]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [40]:
#vendido é nossa classe, e as outras variáveis (features) que vamos analisar.

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify=y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [41]:
#antes de rodar o classificador, precisamos decidir o que é uma taxa de acerto aceitável, boa e ruim.
#o dummy classifier é estratificado, ele pega o que temos de experiência e usa isso.

from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


In [42]:
#vamos rodar uma árvore de decisão

from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 71.92%


In [43]:
# observando a diferença da acurácia de acordo com o SEED utilizado
# não podemos basear uma decisão numa aleatoriedade e precisamos minimizar esse risco
# precisamos treinar mais de uma vez o modelo para ter uma estimativa (um intervalo)

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify=y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 76.84%


In [44]:
# cross validation - separa os dados em k pedaços e rodar k testes com cada um deles e aí faz a média dos resultados (k fold)
# como a média é um ponto que não necessariamente descreve bem os dados, usamos um intervalo
# escolher um número k (leave one out) pode ser mais lento o processamento, mas entrega uma medida mais realista.

from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = 3, return_train_score= False)
media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [74.99, 76.57]


In [45]:
# com a cross validation a gente tira o peso do SEED, porém ainda temos a influência do cv = 3 - quantidade de pedaços
# de acordo com papers, uma escolha de k entre 5 e 10 já é o suficiente.

SEED = 301
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = 5, return_train_score= False)
media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


# Aleatoriedade no cross validade

O cross_validate() não recebe o parâmetro de aleatoriedade, e esse é o padrão. Da maneira utilizada por nós, ele é determinístico, e nós sabemos em quantos pedaços ele quebrará os dados, porque somos nós quem decidimos essa quantidade. Portanto, a realidade é que a aleatoriedade (random) só é aplicada em DecisionTreeClassifier(). E, por padrão, ele só a usará em um caso específico e raro. Por estas razões, mesmo que mudemos constantemente o valor de SEED, obteremos basicamente sempre os mesmos números. <br>


In [46]:
def imprime_resultados(resultados):
    media = resultados['test_score'].mean()
    desvio_padrao = resultados['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [47]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 75.78
Accuracy intervalo: [74.37, 77.19]


In [48]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x, y, cv = cv, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 75.76
Accuracy intervalo: [73.26, 78.26]


# Simular situação horrível de azar

Pode ser "azar" como pode ser uma proporção de exemplos desbalanceados entre as classes.

In [49]:
dados_azar = dados.sort_values("vendido", ascending= True)

x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]

dados_azar.head()


,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [50]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 57.84
Accuracy intervalo: [34.29, 81.39]


In [51]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 75.78
Accuracy intervalo: [72.30, 79.26]


In [52]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 75.78
Accuracy intervalo: [73.55, 78.01]


# Validação cruzada usando grupos

Gerando dados aleatórios de modelo de carro para simulação de agrupamenteo ao usar o estimador


In [53]:
# é preciso tomar cuidado quando formos separar treino e teste, quando nós testamos, temos que ser capazes de testar como se fosse no mundo real
# se nos treinos e testes temos repetições, é possível que o algoritmo não generalize bem para novos carros, pacientes, etc.
# na hora de separar treino e teste, teríamos que agrupar por modelos e pacientes - então pegamos uma CARACTERÍSTICA DE AGRUPAMENTO e tentamos trabalhar nos nossos dados.

np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [54]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1], dtype=int64)

In [55]:
dados.modelo.value_counts()

modelo
20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: count, dtype: int64

## Testando a validação cruzada com GroupKFold

In [56]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)


cv = StratifiedKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 75.78
Accuracy intervalo: [73.83, 77.73]


## Cross Validation com StandardScaler

In [71]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

#o modelo não usa o seed no scaler, mas o SVC sim
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 77.48%


In [72]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
resultados = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 77.27
Accuracy intervalo: [74.35, 80.20]


In [73]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [74]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
resultados = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 76.70
Accuracy intervalo: [74.30, 79.10]


In [77]:
# o fit() acontece mais de uma vez porque temos dez separações baseadas nos grupos, para treinar e testar
# o que acontece então é um processo de duas fases: 1- pré-processamento que escala e, 2- validação
# esse processo deve ser rodado várias vezes, de acordo com nossos grupos e splits por isso criamos a sequência de passos (pipeline)
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits=10)
resultados = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score= False)
imprime_resultados(resultados)

Accuracy médio: 76.68
Accuracy intervalo: [74.28, 79.08]


## Treinando o modelo final

A ideia é que a validação cruzada num conjunto de dados somente te diz o que você pode esperar. Ela não treina o seu modelo final.<br>
O resultado do cross_validate indica o que você deve esperar. Para obter o modelo final, basta treiná-lo em todos os dados que você usou para o crossvalidation e depois o modelo estará pronto para usar com o predict e classificar dados novos.
